<a href="https://colab.research.google.com/github/avantikalohiyaa/ASSIGNMENT-5/blob/main/assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error





In [25]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ID = test['Id']

In [26]:
train['Source'] = 1
test['Source'] = 0
test['SalePrice'] = np.nan

In [27]:
df = pd.concat([train, test], ignore_index=True)

In [28]:
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [29]:
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    mode = df[col].mode()
    if not mode.empty:
        df[col] = df[col].fillna(mode[0])
    else:
        df[col] = df[col].fillna("Missing")

In [30]:
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
df['GarageAge'] = df['YrSold'] - df['GarageYrBlt']
df['GarageAge'] = df['GarageAge'].fillna(df['GarageAge'].median())
df['IsRemodeled'] = (df['YearBuilt'] != df['YearRemodAdd']).astype(int)


In [31]:
for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col])

In [32]:
df.drop('Id', axis=1, inplace=True)

In [33]:
train_cleaned = df[df['Source'] == 1].drop(['Source'], axis=1)
test_cleaned = df[df['Source'] == 0].drop(['Source', 'SalePrice'], axis=1)

In [34]:
train_cleaned.to_csv("train_cleaned.csv", index=False)
test_cleaned.to_csv("test_cleaned.csv", index=False)

In [35]:
df = pd.read_csv("train_cleaned.csv")
print(df.shape)


(1460, 83)


In [36]:
X = train_cleaned.drop("SalePrice", axis=1)
y = train_cleaned["SalePrice"]

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [39]:
val_preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print(f"Validation RMSE: {rmse:.2f}")

Validation RMSE: 27510.07


In [40]:
test_cleaned = pd.read_csv("test_cleaned.csv")
test_preds = model.predict(test_cleaned)


In [19]:
original_test = pd.read_csv("test.csv")


In [41]:
original_test = pd.read_csv("test.csv")
test_cleaned = pd.read_csv("test_cleaned.csv")
test_preds = model.predict(test_cleaned)

important_features = ['GrLivArea', 'OverallQual', 'TotalSF', 'GarageCars', 'FullBath', 'YearBuilt']
final_output = pd.DataFrame({
    'Id': original_test['Id'],
    'GrLivArea': original_test['GrLivArea'],
    'OverallQual': original_test['OverallQual'],
    'TotalSF': test_cleaned['TotalSF'],
    'GarageCars': original_test['GarageCars'],
    'FullBath': original_test['FullBath'],
    'YearBuilt': original_test['YearBuilt'],
    'PredictedPrice': test_preds
})

In [42]:
final_output.to_csv("house_price_predictions_table.csv", index=False)